# Sequence comparison of ENSP 8861 with shared ENSG_ENST_ENSP KEY ID

## HEAT map file made for # proSequences different

## accounting of 49 Uniprot canonical sequences with no identical match in any ensembl release



v1 markdown = 19_09_11_Ensembl_versionIDs_IDENTITYproseq_MISMAP_SCORING.ipynb


In [1]:
# packages
import os
import sys
import numpy as np
import pandas as pd
import csv
import argparse
import Bio
from Bio import SeqIO
from ast import literal_eval # for mismap_score func
from statistics import mean
import jellyfish
import textdistance

In [2]:
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000
display(HTML("<style>.container {width:90% !important;}</style>"))
sys.path.append("/Users/mariapalafox/Desktop/Toolbox")
from all_funx import *

In [5]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/MULTIMAPPING_IDS/")

---
---
---

# comparing  ENSP IDs shared between all releases, do they all have the same prosequence? does 85 have the most difference compared to other GRCh38 sequences? 
- 85 has most difference
- all ensp IDs have at least 1 different protein sequence with another release

In [7]:
# read in files saved above
enspv85 = pd.read_csv("ENSEMBL_SEQ_v85_3953_UKBIDs.csv")  
enspv92 = pd.read_csv("ENSEMBL_SEQ_v92_3953_UKBIDs.csv")  
enspv94 = pd.read_csv("ENSEMBL_SEQ_v94_3953_UKBIDs.csv")  
enspv96 = pd.read_csv("ENSEMBL_SEQ_v96_3953_UKBIDs.csv")  
enspv97 = pd.read_csv("ENSEMBL_SEQ_v97_3953_UKBIDs.csv")

In [40]:
dfs = [enspv85,enspv92,enspv94,enspv96,enspv97]
for i in dfs: 
    print(i.shape)

(10183, 21)
(10395, 21)
(10612, 21)
(10663, 21)
(10564, 21)


In [14]:
s85 = enspv85[enspv85['key_8861'] == True].copy()
s92 = enspv92[enspv92['key_8861'] == True].copy()
s94 = enspv94[enspv94['key_8861'] == True].copy()
s96 = enspv96[enspv96['key_8861'] == True].copy()
s97 = enspv97[enspv97['key_8861'] == True].copy()

In [19]:
outdf = [s85, s92, s94, s96, s97]
for i in outdf: 
    #dropUnnamed(i)
    i.reset_index(inplace=True,drop=True)

In [26]:
s85.head(3)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861
0,ENSP00000000233.5,ENSP00000000233,ENST00000000233.5,ENST00000000233,ENSG00000004059.6,ENSG00000004059,180,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,ENSG00000004059_ENST00000000233_ENSP00000000233,P84085,5,5,6,True,180,0,0.0,0.0,0.0,0.0,True
1,ENSP00000001008.4,ENSP00000001008,ENST00000001008.4,ENST00000001008,ENSG00000004478.5,ENSG00000004478,459,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...,ENSG00000004478_ENST00000001008_ENSP00000001008,Q02790,4,4,5,True,459,0,0.0,0.0,0.0,0.0,True
2,ENSP00000003302.4,ENSP00000003302,ENST00000003302.4,ENST00000003302,ENSG00000048028.7,ENSG00000048028,1077,MTAELQQDDAAGAADGHGSSCQMLLNQLREITGIQDPSFLHEALKA...,ENSG00000048028_ENST00000003302_ENSP00000003302,Q96RU2,4,4,7,True,1077,0,0.0,0.0,0.0,0.0,True


In [22]:
test85_92 = s85.where(s85.proSequence==s92.proSequence).notna()
checkColumnValues(test85_92,'proSequence')

   proSequence  Count
0         True   8729
1        False    132


In [23]:
test85_94 = s85.where(s85.proSequence==s94.proSequence).notna()
checkColumnValues(test85_94,'proSequence')

   proSequence  Count
0         True   8728
1        False    133


In [24]:
test85_96 = s85.where(s85.proSequence==s96.proSequence).notna()
checkColumnValues(test85_96,'proSequence')

   proSequence  Count
0         True   8710
1        False    151


In [25]:
test85_97 = s85.where(s85.proSequence==s97.proSequence).notna()
checkColumnValues(test85_97,'proSequence')

   proSequence  Count
0         True   8699
1        False    162


# creating dictionary from ENSP and Sequence columns in all release files
**more robust way of testing differences**
## all have the same ENSG_ENST_ENSP IDs - 8861

In [27]:
dict_85 = dict(zip(s85.stableID_key, s85.proSequence))
dict_92 = dict(zip(s92.stableID_key, s92.proSequence))
dict_94 = dict(zip(s94.stableID_key, s94.proSequence))
dict_96 = dict(zip(s96.stableID_key, s96.proSequence))
dict_97 = dict(zip(s97.stableID_key, s97.proSequence))

In [35]:
def Average(ls): 
    avg =  mean(ls) 
    finalavg = round(avg,2)
    return finalavg

def identicalSequences_inEnsembl(dfEnsp, ref_dic, newcolresult):
    res = []
    hamN = []
    levN = []
    serSeq = dfEnsp['proSequence'].copy()
    serID = dfEnsp['stableID_key'].copy()
    for inx, val in serSeq.items():
        pep = str(val)
        p = pep.strip()
        idd = str(serID[inx])
        # check pep to dict pep sequence
        mypep = ref_dic[idd]
        str(mypep)
        #mypep = mypep.strip()
        # identical
        if mypep == p:
            res.append('True')
        # not identical to other release ENSP
        # only does score for different sequences
        if mypep != p: 
            res.append('False')
            # calc. distance scores for enesembl differing sequences
            # normalized hamming = # mismatched positions/ len of longer sequence
            hamN.append(textdistance.hamming.normalized_distance(mypep, p))
            # levenshtein score is edit based but not not penalized position, insertion at pos 1 is jsut 1 diff
            levN.append(textdistance.levenshtein.normalized_distance(mypep,p))
    
    # add result col of seq match
    dfEnsp.loc[:,newcolresult] = res
    # select only False to matching seq rows
    dfFalse = dfEnsp[dfEnsp[newcolresult] == 'False']
    dfFALSE_list = dfFalse['ENSP'].tolist()
    print("ENSP IDs that have different protein sequences btw releases: ", dfFALSE_list)
    checkColumnValues(dfEnsp, newcolresult)
    print("hamming Normalized score average of different ensp sequences: ", Average(hamN))
    print("levenshtein Normalized score average of different ensp sequences: ", Average(levN))
    print()
    #print("returns False for pro seq match dataframe")
    return dfFalse

In [33]:
result_92vs85 = identicalSequences_inEnsembl(s92, dict_85, 'proSeq_sameas85')
result_94vs85 = identicalSequences_inEnsembl(s94, dict_85, 'proSeq_sameas85')
result_96vs85 = identicalSequences_inEnsembl(s96, dict_85, 'proSeq_sameas85')
result_97vs85 = identicalSequences_inEnsembl(s97, dict_85, 'proSeq_sameas85')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000339247', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345881', 'ENSP00000349486', 'ENSP00000350199', 'ENSP00000350563', 'ENSP00000350846', 'ENSP00000351159', 'ENSP00000351483', 'ENSP00000352575', 'ENSP00000353483', 'ENSP00000354907', 'ENSP00000357679', 'ENSP00000358832', 'ENSP0

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000228284', 'ENSP00000229854', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000251527', 'ENSP00000256578', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000263741', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000265537', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000286091', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000300933', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000338806', 'ENSP00000339247', 'ENSP00000340824', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345230', 'ENSP00000345881', 'ENSP0

In [34]:
result_85vs92 = identicalSequences_inEnsembl(s85, dict_92, 'proSeq_sameas92')
result_94vs92 = identicalSequences_inEnsembl(s94, dict_92, 'proSeq_sameas92')
result_96vs92 = identicalSequences_inEnsembl(s96, dict_92, 'proSeq_sameas92')
result_97vs92 = identicalSequences_inEnsembl(s97, dict_92, 'proSeq_sameas92')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000339247', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345881', 'ENSP00000349486', 'ENSP00000350199', 'ENSP00000350563', 'ENSP00000350846', 'ENSP00000351159', 'ENSP00000351483', 'ENSP00000352575', 'ENSP00000353483', 'ENSP00000354907', 'ENSP00000357679', 'ENSP00000358832', 'ENSP0

In [36]:
result_96vs94 = identicalSequences_inEnsembl(s96, dict_94, 'proSeq_sameas94')
result_97vs94 = identicalSequences_inEnsembl(s97, dict_94, 'proSeq_sameas94')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000228284', 'ENSP00000229854', 'ENSP00000251527', 'ENSP00000265537', 'ENSP00000286091', 'ENSP00000338806', 'ENSP00000347847', 'ENSP00000358106', 'ENSP00000372326', 'ENSP00000377343', 'ENSP00000402425', 'ENSP00000404920', 'ENSP00000408108', 'ENSP00000408429', 'ENSP00000422753', 'ENSP00000430868', 'ENSP00000450560', 'ENSP00000455628']
  proSeq_sameas94  Count
0            True   8843
1           False     18
hamming Normalized score average of different ensp sequences:  0.83
levenshtein Normalized score average of different ensp sequences:  0.25

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000228284', 'ENSP00000229854', 'ENSP00000251527', 'ENSP00000256578', 'ENSP00000263741', 'ENSP00000265537', 'ENSP00000286091', 'ENSP00000300933', 'ENSP00000338806', 'ENSP00000340824', 'ENSP00000345230', 'ENSP00000347847', 'ENSP00000353094', 'ENSP00000358106', 'ENSP00000362773', 'ENSP00000363624', 'ENSP0000036

In [37]:
result_97vs96 = identicalSequences_inEnsembl(s97, dict_96, 'proSeq_sameas96')
result_96vs97 = identicalSequences_inEnsembl(s96, dict_97, 'proSeq_sameas97')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000256578', 'ENSP00000263741', 'ENSP00000300933', 'ENSP00000340824', 'ENSP00000345230', 'ENSP00000353094', 'ENSP00000362773', 'ENSP00000363624', 'ENSP00000367332', 'ENSP00000436541', 'ENSP00000441973']
  proSeq_sameas96  Count
0            True   8850
1           False     11
hamming Normalized score average of different ensp sequences:  0.85
levenshtein Normalized score average of different ensp sequences:  0.18

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000256578', 'ENSP00000263741', 'ENSP00000300933', 'ENSP00000340824', 'ENSP00000345230', 'ENSP00000353094', 'ENSP00000362773', 'ENSP00000363624', 'ENSP00000367332', 'ENSP00000436541', 'ENSP00000441973']
  proSeq_sameas97  Count
0            True   8850
1           False     11
hamming Normalized score average of different ensp sequences:  0.85
levenshtein Normalized score average of different ensp sequences:  0.18



# all output saved to AllENSPcommon8861_proSequence_differences.txt
    - dir_M/TSV../AllENSPcommon8861_proSequence_differences.txt

---
---
---

# are uniprot canonical sequences represented by at least 1 ENSP seq in each release?

## save the ENSP IDs that do not have a canonical match in each release, save ENSP IDs that do not have canonical match in ALL releases

In [42]:
# read in files saved above
v85 = pd.read_csv("ENSEMBL_SEQ_v85_3953_UKBIDs.csv")  
v92 = pd.read_csv("ENSEMBL_SEQ_v92_3953_UKBIDs.csv")  
v94 = pd.read_csv("ENSEMBL_SEQ_v94_3953_UKBIDs.csv")  
v96 = pd.read_csv("ENSEMBL_SEQ_v96_3953_UKBIDs.csv")  
v97 = pd.read_csv("ENSEMBL_SEQ_v97_3953_UKBIDs.csv")

In [44]:
v85.head(3)

,Unnamed: 0,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861
0,0,ENSP00000000233.5,ENSP00000000233,ENST00000000233.5,ENST00000000233,ENSG00000004059.6,ENSG00000004059,180,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,ENSG00000004059_ENST00000000233_ENSP00000000233,P84085,5,5,6,True,180,0,0.0,0.0,0.0,0.0,True
1,1,ENSP00000001008.4,ENSP00000001008,ENST00000001008.4,ENST00000001008,ENSG00000004478.5,ENSG00000004478,459,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...,ENSG00000004478_ENST00000001008_ENSP00000001008,Q02790,4,4,5,True,459,0,0.0,0.0,0.0,0.0,True
2,2,ENSP00000003302.4,ENSP00000003302,ENST00000003302.4,ENST00000003302,ENSG00000048028.7,ENSG00000048028,1077,MTAELQQDDAAGAADGHGSSCQMLLNQLREITGIQDPSFLHEALKA...,ENSG00000048028_ENST00000003302_ENSP00000003302,Q96RU2,4,4,7,True,1077,0,0.0,0.0,0.0,0.0,True


In [43]:
dflivt = [v85, v92, v94, v96, v97]
for i in dflivt:
    sortbycol(i, 'ENSP')

In [45]:
v85ALL_ukbset = set(v85['ID'].tolist())
v92ALL_ukbset = set(v92['ID'].tolist())
v94ALL_ukbset = set(v94['ID'].tolist())
v96ALL_ukbset = set(v96['ID'].tolist())
v97ALL_ukbset = set(v97['ID'].tolist())

In [46]:
print(len(v85ALL_ukbset))
print(len(v92ALL_ukbset))
print(len(v94ALL_ukbset))
print(len(v96ALL_ukbset))
print(len(v97ALL_ukbset))

3953
3953
3953
3953
3953


In [47]:
v97ALL_ukbset == v85ALL_ukbset

True

In [48]:
all3953 = list(v97ALL_ukbset)

In [49]:
v85TRUE = v85[v85['identical_2UKBseq'] == True].copy()
v92TRUE = v92[v92['identical_2UKBseq'] == True].copy()
v94TRUE = v94[v94['identical_2UKBseq'] == True].copy()
v96TRUE = v96[v96['identical_2UKBseq'] == True].copy()
v97TRUE = v97[v97['identical_2UKBseq'] == True].copy()

In [50]:
v85TRUE.columns

Index(['Unnamed: 0', 'ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG',
       'Length', 'proSequence', 'stableID_key', 'ID', 'pro_ver', 'tx_ver',
       'gen_ver', 'identical_2UKBseq', 'Length_UKB', 'len_ensp_minus_ukb',
       'hamming_distance', 'hamming_normalized_dist', 'levenshtein_distance',
       'levenshtein_normalized_dist', 'key_8861'],
      dtype='object')

In [51]:
v85TRUE_ukbset = set(v85TRUE['ID'].tolist())
v92TRUE_ukbset = set(v92TRUE['ID'].tolist())
v94TRUE_ukbset = set(v94TRUE['ID'].tolist())
v96TRUE_ukbset = set(v96TRUE['ID'].tolist())
v97TRUE_ukbset = set(v97TRUE['ID'].tolist())

In [52]:
print(len(v85TRUE_ukbset))
print(len(v92TRUE_ukbset))
print(len(v94TRUE_ukbset))
print(len(v96TRUE_ukbset))
print(len(v97TRUE_ukbset))
v85TRUE_ukbset = list(v85TRUE_ukbset)
v92TRUE_ukbset = list(v92TRUE_ukbset)
v94TRUE_ukbset = list(v94TRUE_ukbset)
v96TRUE_ukbset = list(v96TRUE_ukbset)
v97TRUE_ukbset = list(v97TRUE_ukbset)

3861
3853
3869
3865
3860


In [53]:
print(len(v85TRUE_ukbset) - 3953)
print(len(v92TRUE_ukbset) - 3953)
print(len(v94TRUE_ukbset) - 3953)
print(len(v96TRUE_ukbset) - 3953)
print(len(v97TRUE_ukbset) - 3953)

-92
-100
-84
-88
-93


In [ ]:
# which uniprot IDs do not have canonical seq match from each release
def diffList(L1, L2):
    return list(set(L1) - set(L2))

In [54]:
print(len(diffList(all3953,v85TRUE_ukbset)))
print(len(diffList(all3953,v92TRUE_ukbset)))
print(len(diffList(all3953,v94TRUE_ukbset)))
print(len(diffList(all3953,v96TRUE_ukbset)))
print(len(diffList(all3953,v97TRUE_ukbset)))
v85_nomatch = diffList(all3953,v85TRUE_ukbset)
v92_nomatch = diffList(all3953,v92TRUE_ukbset)
v94_nomatch = diffList(all3953,v94TRUE_ukbset)
v96_nomatch = diffList(all3953,v96TRUE_ukbset)
v97_nomatch = diffList(all3953,v97TRUE_ukbset)

different number of IDs :  92
92
different number of IDs :  100
100
different number of IDs :  84
84
different number of IDs :  88
88
different number of IDs :  93
93
different number of IDs :  92
different number of IDs :  100
different number of IDs :  84
different number of IDs :  88
different number of IDs :  93


In [55]:
# what uniprot IDs are not in any of the releases as 100% identical to canonical
shared = set(v85_nomatch) & set(v92_nomatch) & set(v94_nomatch) & set(v96_nomatch) & set(v97_nomatch)
len(shared)

49

In [56]:
shared = list(shared)
# these IDs should not be in the True files from each release at all 
shared

['Q14135',
 'Q15170',
 'O14965',
 'O43708',
 'Q9BZ29',
 'O15061',
 'Q9BX63',
 'Q8IY17',
 'Q86TG7',
 'P11532',
 'Q8NBJ7',
 'A6NNF4',
 'P07686',
 'Q9UMY4',
 'P36639',
 'Q8NCA5',
 'Q6PKG0',
 'P11586',
 'P02765',
 'Q99729',
 'Q9UNH6',
 'Q9BV68',
 'Q68E01',
 'O15392',
 'Q9UM54',
 'Q8WWI1',
 'Q12912',
 'Q9UNH7',
 'Q96ME1',
 'A0FGR8',
 'Q13459',
 'P18887',
 'Q14558',
 'P30837',
 'Q9NRG7',
 'Q9Y520',
 'P53990',
 'Q9Y679',
 'P16278',
 'Q8NBT2',
 'P17927',
 'P20839',
 'Q9UJ41',
 'Q03001',
 'Q9P2N6',
 'Q8WX93',
 'O75400',
 'O60645',
 'P11182']

In [57]:
# Checking if shared uniprot ID (not in True release set) exists in list  
# QC
def canonicalMatchNotFound(els, sharedls):
    for id in sharedls:
        ukbid = id
        if ukbid in els:
            print ("Element Exists")
    print("done checking list")

In [58]:
canonicalMatchNotFound(v85TRUE_ukbset, shared)
canonicalMatchNotFound(v92TRUE_ukbset, shared)
canonicalMatchNotFound(v94TRUE_ukbset, shared)
canonicalMatchNotFound(v96TRUE_ukbset, shared)
canonicalMatchNotFound(v97TRUE_ukbset, shared)

done checking list
done checking list
done checking list
done checking list
done checking list


In [59]:
shared

['Q14135',
 'Q15170',
 'O14965',
 'O43708',
 'Q9BZ29',
 'O15061',
 'Q9BX63',
 'Q8IY17',
 'Q86TG7',
 'P11532',
 'Q8NBJ7',
 'A6NNF4',
 'P07686',
 'Q9UMY4',
 'P36639',
 'Q8NCA5',
 'Q6PKG0',
 'P11586',
 'P02765',
 'Q99729',
 'Q9UNH6',
 'Q9BV68',
 'Q68E01',
 'O15392',
 'Q9UM54',
 'Q8WWI1',
 'Q12912',
 'Q9UNH7',
 'Q96ME1',
 'A0FGR8',
 'Q13459',
 'P18887',
 'Q14558',
 'P30837',
 'Q9NRG7',
 'Q9Y520',
 'P53990',
 'Q9Y679',
 'P16278',
 'Q8NBT2',
 'P17927',
 'P20839',
 'Q9UJ41',
 'Q03001',
 'Q9P2N6',
 'Q8WX93',
 'O75400',
 'O60645',
 'P11182']

## testing 1 Uniprot ID that is 'notin97' release fasta

In [60]:
ucan = 'MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFKYSHPHHFLKTTAALRGQVVQFKLSDIGEGIREVTVKEWYVKEGDTVSQFDSICEVQSDKASVTITSRYDGVIKKLYYNLDDIAYVGKPLVDIETEALKDSEEDVVETPAVSHDEHTHQEIKGRKTLATPAVRRLAMENNIKLSEVVGSGKDGRILKEDILNYLEKQTGAILPPSPKVEIMPPPPKPKDMTVPILVSKPPVFTGKDKTEPIKGFQKAMVKTMSAALKIPHFGYCDEIDLTELVKLREELKPIAFARGIKLSFMPFFLKAASLGLLQFPILNASVDENCQNITYKASHNIGIAMDTEQGLIVPNVKNVQICSIFDIATELNRLQKLGSVGQLSTTDLTGGTFTLSNIGSIGGTFAKPVIMPPEVAIGALGSIKAIPRFNQKGEVYKAQIMNVSWSADHRVIDGATMSRFSNLWKSYLENPAFMLLDLK'
print(len(ucan))

482


In [61]:
P11182_v97 = "MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFKYSHPHHFLKTTAALRGQVVQFKLSDIGEGIREVTVKEWYVKEGDTVSQFDSICEVQSDKASVTITSRYDGVIKKLYYNLDDIAYVGKPLVDIETEALKDSEEDVVETPAVSHDEHTHQEIKGRKTLATPAVRRLAMENNIKLSEVVGSGKDGRILKEDILNYLEKQTGAILPPSPKVEIMPPPPKPKDMTVPILVSKPPVFTGKDKTEPIKGFQKAMVKTMSAALKIPHFGYCDEIDLTELVKLREELKPIAFARGIKLSFMPFFLKAASLGLLQFPILNASVDENCQNITYKASHNIGIAMDTEQGLIVPNVKNVQICSIFDIATELNRLQKLGSVSQLSTTDLTGGTFTLSNIGSIGGTFAKPVIMPPEVAIGALGSIKAIPRFNQKGEVYKAQIMNVSWSADHRVIDGATMSRFSNLWKSYLENPAFMLLDLK"

P11182uniprot = "MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFKYSHPHHFLKTTAALRGQVVQFKLSDIGEGIREVTVKEWYVKEGDTVSQFDSICEVQSDKASVTITSRYDGVIKKLYYNLDDIAYVGKPLVDIETEALKDSEEDVVETPAVSHDEHTHQEIKGRKTLATPAVRRLAMENNIKLSEVVGSGKDGRILKEDILNYLEKQTGAILPPSPKVEIMPPPPKPKDMTVPILVSKPPVFTGKDKTEPIKGFQKAMVKTMSAALKIPHFGYCDEIDLTELVKLREELKPIAFARGIKLSFMPFFLKAASLGLLQFPILNASVDENCQNITYKASHNIGIAMDTEQGLIVPNVKNVQICSIFDIATELNRLQKLGSVGQLSTTDLTGGTFTLSNIGSIGGTFAKPVIMPPEVAIGALGSIKAIPRFNQKGEVYKAQIMNVSWSADHRVIDGATMSRFSNLWKSYLENPAFMLLDLK"

In [62]:
P11182_v97 == P11182uniprot

False

In [63]:
print(len(P11182_v97))

482


In [64]:
print(len(P11182uniprot))

482


In [65]:
import difflib
output_list = [li for li in difflib.ndiff(P11182_v97, P11182uniprot) if li[0] != ' ']
output_list

['- S', '+ G']

# checking from 49 list of uniprot IDs with no canonical match in any releases

In [67]:
test = 'A6NNF4'
v97[v97['ID']== test]
# not in 97

,Unnamed: 0,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861
8177,8177,ENSP00000432583.2,ENSP00000432583,ENST00000531821.6,ENST00000531821,ENSG00000213967.10,ENSG00000213967,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,ENSG00000213967_ENST00000531821_ENSP00000432583,A6NNF4,2,6,10,False,738,-661,662.0,0.897019,661.0,0.895664,True
9476,9476,ENSP00000471516.1,ENSP00000471516,ENST00000594466.5,ENST00000594466,ENSG00000213967.10,ENSG00000213967,616,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,ENSG00000213967_ENST00000594466_ENSP00000471516,A6NNF4,1,5,10,False,738,-122,122.0,0.165312,122.0,0.165312,False


In [68]:
v96[v96['ID']== test]
# no identical match in 96

,Unnamed: 0,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861
8242,8242,ENSP00000432583.2,ENSP00000432583,ENST00000531821.6,ENST00000531821,ENSG00000213967.10,ENSG00000213967,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,ENSG00000213967_ENST00000531821_ENSP00000432583,A6NNF4,2,6,10,False,738,-661,662.0,0.897019,661.0,0.895664,True
9549,9549,ENSP00000471516.1,ENSP00000471516,ENST00000594466.5,ENST00000594466,ENSG00000213967.10,ENSG00000213967,616,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,ENSG00000213967_ENST00000594466_ENSP00000471516,A6NNF4,1,5,10,False,738,-122,122.0,0.165312,122.0,0.165312,False


In [69]:
v94[v94['ID']== test]

,Unnamed: 0,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861
8247,8247,ENSP00000432583.2,ENSP00000432583,ENST00000531821.6,ENST00000531821,ENSG00000213967.10,ENSG00000213967,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,ENSG00000213967_ENST00000531821_ENSP00000432583,A6NNF4,2,6,10,False,738,-661,662.0,0.897019,661.0,0.895664,True
9552,9552,ENSP00000471516.1,ENSP00000471516,ENST00000594466.5,ENST00000594466,ENSG00000213967.10,ENSG00000213967,616,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,ENSG00000213967_ENST00000594466_ENSP00000471516,A6NNF4,1,5,10,False,738,-122,122.0,0.165312,122.0,0.165312,False


In [70]:
v92[v92['ID']== test]

,Unnamed: 0,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861
8232,8232,ENSP00000432583.2,ENSP00000432583,ENST00000531821.6,ENST00000531821,ENSG00000213967.10,ENSG00000213967,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,ENSG00000213967_ENST00000531821_ENSP00000432583,A6NNF4,2,6,10,False,738,-661,662.0,0.897019,661.0,0.895664,True
9528,9528,ENSP00000471516.1,ENSP00000471516,ENST00000594466.5,ENST00000594466,ENSG00000213967.10,ENSG00000213967,616,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,ENSG00000213967_ENST00000594466_ENSP00000471516,A6NNF4,1,5,10,False,738,-122,122.0,0.165312,122.0,0.165312,False


In [71]:
v85[v85['ID']== test]
# v85 has ENSP with same length but differs by 1 substituion

,Unnamed: 0,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861
2268,2268,ENSP00000317125.7,ENSP00000317125,ENST00000322487.7,ENST00000322487,ENSG00000213967.6,ENSG00000213967,738,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,ENSG00000213967_ENST00000322487_ENSP00000317125,A6NNF4,7,7,6,False,738,0,1.0,0.001355,1.0,0.001355,False
8507,8507,ENSP00000432583.2,ENSP00000432583,ENST00000531821.2,ENST00000531821,ENSG00000213967.6,ENSG00000213967,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,ENSG00000213967_ENST00000531821_ENSP00000432583,A6NNF4,2,2,6,False,738,-661,662.0,0.897019,661.0,0.895664,True


In [72]:
# for A6NNF4 same length confirming 1 substition diff
v85seq = 'MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIAVSKPDLIICLEKEKEPWNMKRDEMVDEPPGICPHFAQDIWPEQGVEDSFQKVILRRFEKCGHENLQLRKGCKSVDECKVHKEGYNGLNQCFTTTQGKASQCGKYLKVFYKFINLNRYKIRHTRKKPFKCKNCVKSFCMFSHKTQHKSIYTTEKSYKCKECGKTFNWSSTLTNHKKTHTEEKPYKCEEYGKAFNQSSNYTTHKVTHTGEKPYKCEECGKAFSQSSTLTIHKRIHTGEKPCKCEECGKAFSQPSALTIHKRMHIGEKPYKCEECGKAFVWSSTLTRHKRLHSGEKPYKCEECAKAFSQFGHLTTHRIIHTGEKPYKCEECGKAFIWPSTLTKHKRIHTGEKPYKCEECGKAFHRSSNLTKHKIIHTGEKPYKCEECGKAFIWSSNLTEHKKIHTREKPYKCEECSKAFSRSSALTTHKRMHTGEKPYKCEECGKAFSQSSTLTAHKIIHTGEKPYKCEECGKAFILSSTLSKHKRIHTGEKPYKCEECGKTFNQSSNLSTHKIIHTGEKPYKCEECGKAFNRSSNLSTHKIIHTGEKPYKCDECGKSFIWSSTLFKHKIIHTGEKPYKCEECGKAFNHSQILLHIRHKRMHTGEKPYKCEECGKSFNLSSTFIKHKVIHTGVKLYKCEECGKVFFWSSALTRHKKIHAGQQPYKWEKIGKAFNQSSHLTTDKITHIGEKSYKCE'
uniprotseq = 'MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIAVSKPDLIICLEKEKEPWNMKRDEMVDEPPGICPHFAQDIWPEQGVEDSFQKVILRRFEKCGHENLQLRKGCKSVDECKVHKEGYNGLNQCFTTTQGKASQCGKYLKVFYKFINLNRYKIRHTRKKPFKCKNCVKSFCMFSHKTQHKSIYTTEKSYKCKECGKTFNWSSTLTNHKKTHTEEKPYKCEEYGKAFNQSSNYTTHKVTHTGEKPYKCEECGKAFSQSSTLTIHKRIHTGEKPCKCEECGKAFSQPSALTIHKRMHIGEKPYKCEECGKAFVWSSTLTRHKRLHSGEKPYKCEECAKAFSQFGHLTTHRIIHTGEKPYKCEECGKAFIWPSTLTKHKRIHTGEKPYKCEECGKAFHRSSNLTKHKIIHTGEKPYKCEECGKAFIWSSNLTEHKKIHTREKPYKCEECSKAFSRSSALTTHKRMHTGEKPYKCEECGKAFSQSSTLTAHKIIHTGEKPYKCEECGKAFILSSTLSKHKRIHTGEKPYKCEECGKTFNQSSNLSTHKIIHTGEKPYKCEECGKAFNRSSNLSTHKIIHTGEKPYKCDECGKSFIWSSTLFKHKRIHTGEKPYKCEECGKAFNHSQILLHIRHKRMHTGEKPYKCEECGKSFNLSSTFIKHKVIHTGVKLYKCEECGKVFFWSSALTRHKKIHAGQQPYKWEKIGKAFNQSSHLTTDKITHIGEKSYKCE'

```bash
grep A6NNF4 MISMAP_SCORE_UniprotFastaCKabund_w_UniprotCYSLYSpositionsLabeled_3979.csv 
3664,A6NNF4,738,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIAVSKPDLIICLEKEKEPWNMKRDEMVDEPPGICPHFAQDIWPEQGVEDSFQKVILRRFEKCGHENLQLRKGCKSVDECKVHKEGYNGLNQCFTTTQGKASQCGKYLKVFYKFINLNRYKIRHTRKKPFKCKNCVKSFCMFSHKTQHKSIYTTEKSYKCKECGKTFNWSSTLTNHKKTHTEEKPYKCEEYGKAFNQSSNYTTHKVTHTGEKPYKCEECGKAFSQSSTLTIHKRIHTGEKPCKCEECGKAFSQPSALTIHKRMHIGEKPYKCEECGKAFVWSSTLTRHKRLHSGEKPYKCEECAKAFSQFGHLTTHRIIHTGEKPYKCEECGKAFIWPSTLTKHKRIHTGEKPYKCEECGKAFHRSSNLTKHKIIHTGEKPYKCEECGKAFIWSSNLTEHKKIHTREKPYKCEECSKAFSRSSALTTHKRMHTGEKPYKCEECGKAFSQSSTLTAHKIIHTGEKPYKCEECGKAFILSSTLSKHKRIHTGEKPYKCEECGKTFNQSSNLSTHKIIHTGEKPYKCEECGKAFNRSSNLSTHKIIHTGEKPYKCDECGKSFIWSSTLFKHKRIHTGEKPYKCEECGKAFNHSQILLHIRHKRMHTGEKPYKCEECGKSFNLSSTFIKHKVIHTGVKLYKCEECGKVFFWSSALTRHKKIHAGQQPYKWEKIGKAFNQSSHLTTDKITHIGEKSYKCE,0.065,0.1477,True,1,{55: 'C'}
```

In [73]:
import difflib
output_list = [li for li in difflib.ndiff(v85seq, uniprotseq) if li[0] != ' ']
output_list
# difference between the 2 sequences is I changed to R in uniprot seq

['- I', '+ R']

# SEE 19_09_12_QC_MISMAP_RESULTS for more quality testing on sequence comparison results

---
---
---

---
---
---